In [ ]:
from __future__ import print_function
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import spartan.utils.utils as spartan_utils
from imitation_agent.dataset.function_factory import ObservationFunctionFactory, ActionFunctionFactory, get_gt_dynamic_object_translation_x
from imitation_agent.dataset.imitation_episode_sequence_dataset import ImitationEpisodeSequenceDataset
from imitation_agent.dataset.imitation_episode_dataset import ImitationEpisodeDataset
from imitation_agent.dataset.directory_structure import SingleEpisodeDirectoryStructure


spartan_source_dir = spartan_utils.getSpartanSourceDir()
logs_config_yaml = os.path.join(spartan_source_dir, "modules/imitation_agent/config/task/push_box.yaml")
logs_config = spartan_utils.getDictFromYamlFilename(logs_config_yaml)
logs_dir_path = "/home/manuelli/data_ssd/imitation/logs/push_box" # set individually

# imitation_src_dir = os.path.join(spartan_source_dir, "modules/imitation_agent")
# data_dir = spartan_utils.get_data_dir()
# logs_dir_path = os.path.join(data_dir, "pdc/imitation/move_to_box_0710")

# config_yaml = os.path.join(imitation_src_dir, "config", "model", "mlp_stateless_position.yaml")
# config = spartan_utils.getDictFromYamlFilename(config_yaml)

# obs_function = ObservationFunctionFactory.get_function(config)
# action_function = ActionFunctionFactory.action_from_config(config)
# dataset = ImitationEpisodeDataset(logs_dir_path, logs_config, config,
#                                   action_function=action_function,
#                                   observation_function=obs_function)

## Helper functions

In [ ]:
## Helper functions
def load_json(experiment_dir, name, index):
    """
    Gets the data_00001.json file and loads it
    """
    index_str = index_str = str(index).zfill(5)
    filename = os.path.join(experiment_dir, experiment_name_dict[name], "data_%s.json" %(index_str))
    return spartan_utils.read_json(filename)

def get_object_dx(path_to_processed_dir):
    """
    For a demonstration run
    """
    ds = SingleEpisodeDirectoryStructure(path_to_processed_dir=path_to_processed_dir)
    json_data = spartan_utils.read_json(ds.state_file)
    N = len(json_data)
    start_data = json_data[str(0)]
    end_data = json_data[str(N-1)]
    
    x_start = get_gt_dynamic_object_translation_x(start_data)
    x_end = get_gt_dynamic_object_translation_x(end_data)
    dx_signed = x_end - x_start
    dx = abs(dx_signed)
    
    return_data = {"x_start": x_start,
                  "x_end": x_end,
                  "dx": dx,
                  "dx_signed": dx_signed}
    
    return return_data

def extract_box_trajectory(json_data, plot=True):
    """
    Extract the box trajectory from the json.
    Optionally plot the results as well
    """
    t = json_data["times"]
    object_pose_list = json_data["object_poses"]
    
    N = len(t)
    obj_pos = np.zeros([N, 3])
    for i in xrange(0, len(t)):
        T = np.array(object_pose_list[i])
        obj_pos[i, :] = T[:3, 3]
        
    print("len(t)", len(t))
    print("len(object_pose_list)", len(object_pose_list))
    print("y_max", np.max(obj_pos[:, 1]))
    if plot:
        plt.plot(t, obj_pos[:, 0])
        plt.ylabel("x position")
        plt.show()

        plt.plot(t, obj_pos[:, 1])
        plt.ylabel("y position")
        plt.show()
        
    return t, obj_pos

def add_data_to_frame(df):
    """
    Compute additional data (such as dx) and add it to the DataFrame
    """
    
    name_list = []
    index_list = []
    dx_list = []
    dy_list = []
    dx_signed_list = []
    
    for _, row in df.iterrows():
        json_data = load_json(experiment_dir, row["name"], row["index"])
        start_pose = np.array(json_data["object_poses"][0])
        end_pose = np.array(json_data["object_poses"][-1])
        pos_delta = end_pose[:3, 3] - start_pose[:3, 3]
        
        dx = abs(pos_delta[0])
        dx_signed = pos_delta[0]        
        dx_list.append(dx)
        dx_signed_list.append(dx_signed)
        
        dy = abs(pos_delta[1])
        dy_list.append(dy)
        
        name_list.append(row["name"])
        index_list.append(row["index"])
    
    data_tmp = {'name': name_list,
               'index': index_list,
               'dx': dx_list,
               'dx_signed': dx_signed_list,
               'dy': dy}
    df_tmp = pd.DataFrame(data_tmp, columns=["name", "index", "dx", "dx_signed", "dy"])
    
    df_merge = pd.merge(df, df_tmp)
    return df_merge, df_tmp


## Load the dataframes

In [ ]:
# experiment_name_dict = {'gt_pose': '01-gt-pose-num-logs-50',
#                        'gt-3D-points': '02-gt-3D-points-50',
#                        'gt-3D-points-projected': '03-gt-3D-points-projected-num-logs-50',
#                        'E2E': '04-endtoend-num-logs-50',
#                        'DD': '05-dd-num-logs-50'}

# experiment_names = ['gt_pose', 'gt-3D-points', 'gt-3D-points-projected', 'E2E', 'DD']

experiment_name_dict = {'gt_pose': '01-gt-pose-num-logs-50',
                       'gt-3D-points': '02-gt-3D-points-50',
                       'gt-3D-points-projected': '03-gt-3D-points-projected-num-logs-50',
                       }


experiment_names = ['gt_pose', 'gt-3D-points', 'gt-3D-points-projected']

experiment_dir = "/home/manuelli/data/pdc/imitation/deploy_evaluation/experiment_03_new"

df_list = []

for name, folder in experiment_name_dict.iteritems():
    df_local = pd.read_csv(os.path.join(experiment_dir, folder, "results.csv"))
    df_local['name'] = name
    df_list.append(df_local)
    print("%s: num experiments %d" %(name, df_local.shape[0]))
    
df = pd.concat(df_list)
print(df.keys())


# create new column by
df["angle_error_degrees"] = np.rad2deg(df["angle_error"])




        
df, df_tmp = add_data_to_frame(df)
len(df_tmp.index)

## Load demonstration data
Store it in the demonstrations_df dataframe

In [ ]:

    
log_name_list = []
dx_list = []
dx_signed_list = []
for log_name in logs_config["logs"]:
    path_to_processed_dir = os.path.join(logs_dir_path, log_name, "processed")
    tmp = get_object_dx(path_to_processed_dir)
    dx = tmp["dx"]
    dx_signed = tmp["dx_signed"]
    log_name_list.append(log_name)
    dx_list.append(dx)
    dx_signed_list.append(dx_signed)
    
data_tmp = {"log_name": log_name_list,
           "dx": dx_list,
           "dx_signed": dx_signed_list}
demonstrations_df = pd.DataFrame(data_tmp, columns=["name", "dx", "dx_signed"])
demonstrations_df["termination_type"] = "STATE"
demonstrations_df["name"] = "demonstrations"

df_combined = pd.concat([df, demonstrations_df])
    

## Print out useful statistics for each run

In [ ]:
name_list = []
state_termination_list = []
time_termination_list = []
unstable_termination_list = []
time_w_dx_nonzero_list = []
for name in experiment_names:
    df_tmp = df[df["name"] == name]
    state_termination = len(df_tmp[df_tmp["termination_type"] == "STATE"])
    time_termination = len(df_tmp[df_tmp["termination_type"] == "TIME"])
    unstable_termination = len(df_tmp[df_tmp["termination_type"] == "UNSTABLE"])
    
    time_w_dx_nonzero = len(df_tmp[(df_tmp["termination_type"] == "TIME") & df_tmp["dx"] > 0.0001])
    
    print("\n\n")
    print("Name:", name)
    print("STATE:", state_termination)
    print("TIME:", time_termination)
    print("UNSTABLE:", unstable_termination)
    print("TIME and dx nonzero:", time_w_dx_nonzero)
    
    name_list.append(name)
    state_termination_list.append(state_termination)
    time_termination_list.append(time_termination)
    unstable_termination_list.append(unstable_termination)
    time_w_dx_nonzero_list.append(time_w_dx_nonzero)

## Termination type bar chart

In [ ]:
# TIME
ax = sns.barplot(x=name_list, y=time_termination_list)
ax.set_xlabel("experiment")
ax.set_ylabel("# termination on TIME")
plt.show()


# TIME
ax = sns.barplot(x=name_list, y=time_w_dx_nonzero_list)
ax.set_xlabel("experiment")
ax.set_ylabel("# termination on TIME and dx > 0")
plt.show()

# STATE
ax = sns.barplot(x=name_list, y=state_termination_list)
ax.set_xlabel("experiment")
ax.set_ylabel("# termination on STATE")
plt.show()

## Angle Error BoxPlot

In [ ]:
## create violin plots of reward
def create_boxplot_angle_error(df):
    sns.set(style="whitegrid")
    ax = sns.boxplot(x=df["name"], y=df["angle_error_degrees"])
    ax.set_ylim(0, 2)
    fig = plt.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
    
create_boxplot_angle_error(df[(df["termination_type"] == "STATE")])

## Delta X from starting location

In [ ]:
## create violin plots of reward
def create_boxplot_dx(df, key="dx"):
    sns.set(style="whitegrid")
#     ax = sns.violinplot(x=df["name"], y=df["reward"])
    df_tmp = df[df["termination_type"] == "STATE"]
    ax = sns.boxplot(x=df_tmp["name"], y=df_tmp[key])
#     ax.set_ylim(-1.0, 0)
    ax.set_ylim(-0.1, 0.1)
    fig = plt.gcf()
    fig.set_size_inches(15, 4)
    plt.show()
 

## compare these to where the ground-truth controller ends up
# require running a bunch of deploys
# create_boxplot_dx(df)
# create_boxplot_dx(demonstrations_df)

df_combined = pd.concat([df, demonstrations_df])
create_boxplot_dx(df_combined)
create_boxplot_dx(df_combined, key="dx_signed")

# DEPRECATED and TESTING

In [ ]:
name = "gt-3D-points-projected"
idx = 36
json_data = get_json_data(name, idx)
t, obj_pos = extract_box_trajectory(json_data, plot=True)

# plot all failures for a given network type
name = 'E2E'
DEPLOY_ROOT_DIR = "/home/manuelli/data/pdc/imitation/deploy_evaluation/experiment_03"

# df_tmp = df[(df["name"] == name) & (df["termination_type"] != "STATE")]
for idx, row in df[(df["name"] == name) & (df["termination_type"] != "STATE")].iterrows():
    
    index = row["index"]
    print("\n\n-----------")
    print("index:", index)
    json_file = os.path.join(DEPLOY_ROOT_DIR, experiment_name_dict[name], "data_%s.json" %(str(index).zfill(5)))
    json_data = spartan_utils.read_json(json_file)
    extract_box_trajectory(json_data)

In [ ]:
ROOT_DIR = "/home/manuelli/data/pdc/imitation/deploy_evaluation/experiment_03/02-gt-3D-points"
for index in xrange(5):
    json_file = os.path.join(ROOT_DIR, "data_%s.json" %(str(index).zfill(5)))
    json_data = spartan_utils.read_json(json_file)
    extract_box_trajectory(json_data)

In [ ]:
df.iloc[0, :]